In [2]:
import pickle
import numpy as np
from metrics import mapk
from collections import defaultdict

In [71]:
# подразумеваются результаты для 2к определенных пользователей, иначе надо добавить проверку по id
class Ensemble():
    def __init__(self, most_popular_prediction, item_based_prediction, user_based_prediction, actual_purchases):
        '''
        :param most_popular_predict: dictionary like {user_id: [predicted products]}
        :param item_based_predict: dictionary like {user_id: [predicted products]}
        :param user_based_predict: dictionary like {user_id: [predicted products]}
        :param actual_purchases: dictionary like {user_id: [actual products]}
        '''
        self.predictions = {'most popular': most_popular_prediction,
                            'item based'  : item_based_prediction,
                            'user based'  : user_based_prediction}
        self.actual = actual_purchases
        
    # AP@k
    def apk(self, actual, predicted, k=10):
        
        if len(predicted) > k:
            predicted = predicted[:k]

        score = 0.0
        num_hits = 0.0

        for i,p in enumerate(predicted):
            if p in actual and p not in predicted[:i]:
                num_hits += 1.0
                score += num_hits / (i+1.0)

        if not actual:
            return 0.0

        return score / min(len(actual), k)
    
    def fit(self, users_ids=False):
        '''
        :param user_ids: list with IDs of interesting users (2k)
        :return: dict {user_id: [products ids from best prediction]}
        '''
        if not users_ids:
            users_ids = [*self.predictions['user based']]

        self.predictions, self.predictions_models = self.get_best_predictions(users_ids=users_ids)
        return self.predictions, self.predictions_models
    
    def get_best_predictions(self, users_ids):
        predictions = {}
        predictions_models = {}
        self.current_users_actual = {}
        
        d = {0: 'most popular', 1: 'item based', 2:'user based'}
        
        for user_id in users_ids:
            if user_id in self.predictions['most popular'] and \
                user_id in self.predictions['item based'] and \
                user_id in self.predictions['user based'] and \
                user_id in self.actual:
                
                # в actual нет юзера с id7, поэтому проверку оставляем
                
                metrics = np.array([self.apk(self.actual[user_id], self.predictions['most popular'][user_id]),
                                    self.apk(self.actual[user_id], self.predictions['item based'][user_id]),
                                    self.apk(self.actual[user_id], self.predictions['user based'][user_id])])
                
                # результатом будет предсказание и название модели, сделавшей его
                predictions_models[user_id] = d[np.argmax(metrics)]
                predictions[user_id] = self.predictions[d[np.argmax(metrics)]][user_id]
                self.current_users_actual[user_id] = self.actual[user_id]
                
        return predictions, predictions_models
    
    def predict(self, user_id=False):
        if user_id:
            return self.predictions[user_id], self.predictions_models[user_id]
        else:
            return self.predictions, self.predictions_models
        
    def get_actual(self):
        return self.current_users_actual

In [53]:
with open('data/item_based_interesting_users.pickle', 'rb') as f:
    item_based_prediction = pickle.load(f)
    
with open('data/most_popular_full.pickle', 'rb') as f:
    most_popular_prediction = pickle.load(f)
    
with open('data/full_actual_purchases.pickle', 'rb') as f:
    actual_purchases = pickle.load(f)

with open('users_subsample.pickle', 'rb') as input:
        users_ids = pickle.load(input)

with open('data/user_based_interesting_users.pickle', 'rb') as f:
    user_based_prediction = pickle.load(f)

In [67]:
# число индексов генерящихся рандомно
k = 0
for _id in users_ids:
    if _id not in item_based_prediction:
        k += 1
k

995

In [72]:
predictor = Ensemble(most_popular_prediction, item_based_prediction, user_based_prediction, actual_purchases)
predictor.fit(users_ids)
predictions = predictor.predict()
current_actual = predictor.get_actual()

In [73]:
# соотношение кто для скольких оказался лучшим
d = defaultdict(int)
for model in predictions[1].values():
    d[model] +=1
d

defaultdict(int, {'item based': 629, 'user based': 154, 'most popular': 137})

In [74]:
# метрички для ансамбля но только для 1000, почему-то тысяча в id генерилась рандомно, пересчитываем пока
current_actual = predictor.get_actual()
print(mapk(current_actual.values(), predictions[0].values(), k=1),
      mapk(current_actual.values(), predictions[0].values(), k=5),
      mapk(current_actual.values(), predictions[0].values(), k=10),
      mapk(current_actual.values(), predictions[0].values(), k=30))

0.6706521739130434 0.4333010265700483 0.3900200872317525 0.36852555786805075


In [75]:
# исходные метрички для user_based

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
user_based_pr = [user_based_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), user_based_pr, k=1),
      mapk(current_actual.values(), user_based_pr, k=5),
      mapk(current_actual.values(), user_based_pr, k=10),
      mapk(current_actual.values(), user_based_pr, k=30))

0.5673913043478261 0.3177530193236715 0.2845707312514377 0.2824318005615539


In [76]:
# исходные метрички для most popular

# чтобы почекать старые пришлось извернуться - там почему-то перекручены ключи и просто зипануть не вышло
most_popular_pr = [most_popular_prediction[_id] for _id in current_actual]

print(mapk(current_actual.values(), most_popular_pr, k=1),
      mapk(current_actual.values(), most_popular_pr, k=5),
      mapk(current_actual.values(), most_popular_pr, k=10),
      mapk(current_actual.values(), most_popular_pr, k=30))

0.08695652173913043 0.10041062801932367 0.09913101536911059 0.09577834099629977
